In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from fbprophet import Prophet
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
feature_data=pd.read_csv('../input/dengai-dataset/DengAI_Predicting_Disease_Spread_-_Training_Data_Features.csv')
feature_data.head()
feature_data.dtypes

In [ ]:
case_data=pd.read_csv('../input/dengai-dataset/DengAI_Predicting_Disease_Spread_-_Training_Data_Labels.csv')
case_data.head()


In [ ]:
case_data.dtypes

In [ ]:
feature_data['week_start_date']=pd.to_datetime(feature_data['week_start_date'],format='%Y-%m-%d')
feature_data['yearweek']= feature_data['year'].astype(str) + feature_data['weekofyear'].astype(str)
case_data['yearweek']= case_data['year'].astype(str) + case_data['weekofyear'].astype(str)


In [ ]:
print(case_data)
case_data.dtypes
feature_data.dtypes
training_data = pd.merge(feature_data,case_data[['city','total_cases','yearweek']], how='left',on=['city','yearweek'])
training_data.head()

In [ ]:
is_sj = training_data.city == 'sj'
sj_training_data=training_data[is_sj]
sj_training_data.head()
sj_training_data.describe()

In [ ]:
plt.scatter(sj_training_data['week_start_date'], sj_training_data['total_cases'])
plt.show()

In [ ]:
sj_training_data.dtypes

In [ ]:
#creating a subset from sj_training_data to be used in fbprohet model
sj_model_data= sj_training_data.iloc[:,[3,25]]
sj_model_data.columns=["ds","y"]
sj_model_data.tail()
sj_prophet=Prophet(yearly_seasonality=True, weekly_seasonality=True, seasonality_mode='multiplicative')
sj_prophet.fit(sj_model_data)
#help(Prophet.make_future_dataframe)
sj_prop_future= sj_prophet.make_future_dataframe(freq='W',periods=260)
sj_prop_future.tail()
sj_prop_forecast= sj_prophet.predict(sj_prop_future)
sj_prop_forecast.tail()
plot1 = sj_prophet.plot(sj_prop_forecast)
plot2 = sj_prophet.plot_components(sj_prop_forecast)

In [ ]:
#help(pd.DataFrame)
sj_test_forecast=pd.DataFrame(dict(city="sj",ds=sj_prop_forecast['ds'], yhat=sj_prop_forecast['yhat']))
sj_test_forecast.tail()
sj_predicted=sj_test_forecast.iloc[936:1196,]
sj_predicted.shape
sj_predicted.columns=["city","ds","total_cases"]
sj_predicted.tail()

In [ ]:
#IQ's analysis
is_iq = training_data.city == 'iq'
iq_training_data=training_data[is_iq]
iq_training_data.head()
iq_training_data.describe()

In [ ]:
iq_model_data= iq_training_data.iloc[:,[3,25]]
iq_model_data.columns=["ds","y"]
iq_model_data.tail()
iq_prophet=Prophet(yearly_seasonality=True, weekly_seasonality=True, seasonality_mode='multiplicative')
iq_prophet.fit(iq_model_data)
#help(Prophet.make_future_dataframe)
iq_prop_future= iq_prophet.make_future_dataframe(freq='W',periods=156)
iq_prop_future.tail()
iq_prop_forecast= iq_prophet.predict(iq_prop_future)
iq_prop_forecast.tail()
plot1 = iq_prophet.plot(iq_prop_forecast)
plot2 = iq_prophet.plot_components(iq_prop_forecast)

In [ ]:
iq_test_forecast=pd.DataFrame(dict(city="iq",ds=iq_prop_forecast['ds'], yhat=iq_prop_forecast['yhat']))
iq_test_forecast.tail()
iq_predicted=iq_test_forecast.iloc[520:676,]
iq_predicted.shape
iq_predicted.columns=["city","ds","total_cases"]
iq_predicted.tail()

In [ ]:
result=sj_predicted.append(iq_predicted)
result.head()
result.shape

In [ ]:
result.to_csv('result-v1.csv')